In [ ]:
!pip install biopython bs4 requests Datasets

In [ ]:
!huggingface-cli login

In [ ]:
import glob
import ftplib
import requests
import subprocess
from bs4 import BeautifulSoup
from itertools import islice
import pandas as pd
from Bio import SeqIO
from datasets import load_dataset
import time
import subprocess
import glob
from Bio import SeqIO
import pandas as pd
import concurrent.futures
import concurrent.futures
import glob
import subprocess

In [ ]:
def batched(iterable, n):
    # batched('ABCDEFG', 3) --> ABC DEF G
    if n < 1:
        raise ValueError('n must be at least one')
    it = iter(iterable)
    while batch := tuple(islice(it, n)):
        yield batch

In [ ]:
def calculate_total_size(url):
    response = requests.get(url)
    response.raise_for_status()  # Ensure we got a valid response
    soup = BeautifulSoup(response.content, 'html.parser')

    # Regular expression to find numbers followed by an 'M'
    pattern = re.compile(r'(\d+(\.\d+)?)M')
    matches = pattern.findall(str(soup))

    # Extract the first item from each tuple to get the numbers
    sizes = [match[0] for match in matches]
    sizes = [float(size) for size in sizes]
    print(sizes)
    return f'total size of files on the site is: {sum(sizes)}'

In [ ]:
def get_links(url):
    # Fetch the content of the page
    response = requests.get(url)
    response.raise_for_status()  # Ensure we got a valid response

    # Parse the page using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all anchor tags (links) and filter those ending with .gz
    gz_links = [link.get('href') for link in soup.find_all('a') if link.get('href', '').endswith('.gz')]
    return gz_links

url = "https://ftp.ncbi.nlm.nih.gov/genbank/"  # Replace this with the website you want to scrape
gz_links = get_links(url)
print(f"The website {url} contains {len(gz_links)} .gz links.")


In [ ]:
delete_parquets = f'rm /content/upload_folder/*.parquet'
delete_sequences = 'rm *.seq'

In [ ]:
def gunzip_file(file):
    """Function to decompress a single file."""
    subprocess.run(f"gunzip {file}", shell=True)

def process_file(file, file_count):
    seqs_data = {
        'id': [],
        'sequence': [],
        'name': [],
        'description': [],
        'features': [],
        'seq_length': []
    }
    try:
        for record in SeqIO.parse(file, "genbank"):
            seqs_data['id'].append(str(record.id))
            seqs_data['sequence'].append(str(record.seq))
            seqs_data['name'].append(str(record.name))
            seqs_data['description'].append(str(record.description))
            seqs_data['features'].append(len(record.features))
            seqs_data['seq_length'].append(len(str(record.seq)))
    except Exception as e:
        print(f'{file} has an error: {e}')

    count = file_count[file]  # Assuming `file_count` is a dict that maps filenames to an index
    df = pd.DataFrame(seqs_data)
    df.to_parquet(f'/content/upload_folder/file_{count}.parquet')
    return f"function call finished for {file}"

def process_batch(batch):
    for link in batch:
        command = f'wget https://ftp.ncbi.nlm.nih.gov/genbank/{link}'
        subprocess.run(command, shell=True)

    subprocess.run(f"{delete_parquets} && {delete_sequences}", shell=True)
    gz_files = glob.glob("**.gz")
    with concurrent.futures.ProcessPoolExecutor() as executor:
        executor.map(gunzip_file, gz_files)

    files = glob.glob("*.seq")
    # Create a mapping from file name to a unique index
    file_count = {f: idx for idx, f in enumerate(files)}

    with concurrent.futures.ProcessPoolExecutor() as executor:
        results = list(executor.map(process_file, files, [file_count]*len(files)))


    for result in results:
        print(result)

    print('loading parquet files')
    dataset = load_dataset("parquet", data_files="/content/upload_folder/*.parquet")
    dataset.push_to_hub(f'Hack90/ncbi_genbank_part_{batch_count}')
    dataset.cleanup_cache_files()
    subprocess.run('rm -rf ~/.cache/huggingface/datasets', shell=True)
batch_count = 1
for batch in batched(gz_links, 100):
    if batch_count > 7:
        process_batch(batch)
    batch_count += 1
